In [ ]:
!mkdir -p /content/vizwiz/{images/train,images/val,images/test,annotations}

In [ ]:
!wget https://vizwiz.cs.colorado.edu/VizWiz_final/images/train.zip \
     -O /content/vizwiz/train.zip

--2025-05-11 09:57:36--  https://vizwiz.cs.colorado.edu/VizWiz_final/images/train.zip
Resolving vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)... 198.59.7.50
Connecting to vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)|198.59.7.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11298421598 (11G) [application/zip]
Saving to: ‘/content/vizwiz/train.zip’

/content/vizwiz/tra 100%[===================>]  10.52G  19.6MB/s    in 10m 31s 

2025-05-11 10:08:08 (17.1 MB/s) - ‘/content/vizwiz/train.zip’ saved [11298421598/11298421598]



In [ ]:
!unzip -q /content/vizwiz/train.zip  -d /content/vizwiz/images/train


In [ ]:
!wget https://vizwiz.cs.colorado.edu/VizWiz_final/images/val.zip \
     -O /content/vizwiz/val.zip


--2025-05-11 10:10:40--  https://vizwiz.cs.colorado.edu/VizWiz_final/images/val.zip
Resolving vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)... 198.59.7.50
Connecting to vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)|198.59.7.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3488913457 (3.2G) [application/zip]
Saving to: ‘/content/vizwiz/val.zip’

/content/vizwiz/val 100%[===================>]   3.25G  9.36MB/s    in 3m 50s  

2025-05-11 10:14:30 (14.5 MB/s) - ‘/content/vizwiz/val.zip’ saved [3488913457/3488913457]



In [ ]:
!unzip -q /content/vizwiz/val.zip    -d /content/vizwiz/images/val

In [ ]:
!wget https://vizwiz.cs.colorado.edu/VizWiz_final/images/test.zip \
     -O /content/vizwiz/test.zip


--2025-05-11 10:15:14--  https://vizwiz.cs.colorado.edu/VizWiz_final/images/test.zip
Resolving vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)... 198.59.7.50
Connecting to vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)|198.59.7.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975272799 (3.7G) [application/zip]
Saving to: ‘/content/vizwiz/test.zip’

/content/vizwiz/tes 100%[===================>]   3.70G  15.6MB/s    in 5m 24s  

2025-05-11 10:20:39 (11.7 MB/s) - ‘/content/vizwiz/test.zip’ saved [3975272799/3975272799]



In [ ]:
!unzip -q /content/vizwiz/test.zip   -d /content/vizwiz/images/test

In [ ]:
!wget https://vizwiz.cs.colorado.edu/VizWiz_final/vqa_data/Annotations.zip \
     -O /content/vizwiz/annotations/annotations.zip
!unzip -q /content/vizwiz/annotations/annotations.zip \
        -d /content/vizwiz/annotations

--2025-05-11 10:21:29--  https://vizwiz.cs.colorado.edu/VizWiz_final/vqa_data/Annotations.zip
Resolving vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)... 198.59.7.50
Connecting to vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)|198.59.7.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1711273 (1.6M) [application/zip]
Saving to: ‘/content/vizwiz/annotations/annotations.zip’

/content/vizwiz/ann 100%[===================>]   1.63M  2.02MB/s    in 0.8s    

2025-05-11 10:21:30 (2.02 MB/s) - ‘/content/vizwiz/annotations/annotations.zip’ saved [1711273/1711273]



In [ ]:
!pip install torch torchvision transformers datasets evaluate pillow


In [ ]:
# 4) Charger manuellement en DatasetDict
from datasets import Dataset, DatasetDict
import json, os

def load_split(ann_file, img_dir):
    with open(ann_file, 'r') as f:
        annotations = json.load(f)
    rows = []
    for item in annotations:
        # Check if 'answers' key exists before accessing it
        answers = item.get("answers", [])
        # If 'answers' key doesn't exist, use an empty list for answers
        rows.append({
            "image": os.path.join(img_dir, item["image"]),
            "question": item["question"],
            "answers": [ans["answer"] for ans in answers]
        })
    return Dataset.from_list(rows)

base = "/content/vizwiz"
imgs = os.path.join(base, "images")
anns = os.path.join(base, "annotations")

dataset = DatasetDict({
    "train": load_split(
        os.path.join(anns, "train.json"),
        os.path.join(imgs, "train", "train")    # ← images/train/train/…
    ),
    "validation": load_split(
        os.path.join(anns, "val.json"),
        os.path.join(imgs, "val",   "val")      # ← images/val/val/…
    ),
    "test": load_split(
        os.path.join(anns, "test.json"),
        os.path.join(imgs, "test",  "test")     # ← images/test/test/…
    ),
})

# 5) Vérifier
print(dataset)
print("Colonnes :", dataset["train"].column_names)
print("Exemple :", dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['image', 'question', 'answers'],
        num_rows: 20523
    })
    validation: Dataset({
        features: ['image', 'question', 'answers'],
        num_rows: 4319
    })
    test: Dataset({
        features: ['image', 'question', 'answers'],
        num_rows: 8000
    })
})
Colonnes : ['image', 'question', 'answers']
Exemple : {'image': '/content/vizwiz/images/train/train/VizWiz_train_00000000.jpg', 'question': "What's the name of this product?", 'answers': ['basil leaves', 'basil leaves', 'basil', 'basil', 'basil leaves', 'basil leaves', 'basil leaves', 'basil leaves', 'basil leaves', 'basil']}


In [ ]:
from transformers import BlipProcessor
from PIL import Image

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")



In [ ]:
def preprocess_fn(example):
    # 1) Charge l'image
    img_path = example["image"]
    image = Image.open(img_path).convert("RGB")

    # 2) Tokenise la question + prépare l'image
    inputs = processor(image, example["question"], return_tensors="pt")

    # 3) Prend la 1ʳᵉ réponse comme label (on tronque/sèche à max_length)
    answer = example["answers"][0]
    labels = processor.tokenizer(
        answer,
        truncation=True,
        padding="max_length",
        max_length=16,
        return_tensors="pt"
    ).input_ids

    # 4) Supprime la dimension batch inutile
    return {
        "pixel_values": inputs.pixel_values.squeeze(0),
        "input_ids":    inputs.input_ids.squeeze(0),
        "attention_mask": inputs.attention_mask.squeeze(0),
        "labels":         labels.squeeze(0)
    }



In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class VQADataset(Dataset):
    def __init__(self, ds, processor, max_length=16):
        self.ds = ds
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]
        img = Image.open(item["image"]).convert("RGB")
        proc = self.processor(
            img,
            item["question"],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        label = self.processor.tokenizer(
            item["answers"][0],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        ).input_ids.squeeze(0)
        return {
            "pixel_values":   proc.pixel_values.squeeze(0),
            "input_ids":      proc.input_ids.squeeze(0),
            "attention_mask": proc.attention_mask.squeeze(0),
            "labels":         label
        }

# Puis :
train_dataset = VQADataset(dataset["train"], processor)
train_loader  = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_dataset = VQADataset(dataset["validation"], processor)
val_loader  = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=2)

In [ ]:
# 1) Choisis combien d’exemples tu veux (par ex. 200)
# n_debug = 1500

# # 2) Crée un nouveau Dataset “petit train” à partir du train complet
# small_train_split = dataset["train"].shuffle(seed=42).select(range(n_debug))

# # 3) Instancie ton Dataset PyTorch dessus
# small_train_dataset = VQADataset(small_train_split, processor)

# # 4) DataLoader sur ce petit train
# from torch.utils.data import DataLoader
# train_loader = DataLoader(
#     small_train_dataset,
#     batch_size=8,     # tu peux aussi réduire la batch size
#     shuffle=True,
#     num_workers=2
# )


In [ ]:
import torch
from transformers import BlipForQuestionAnswering, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm.auto import tqdm  # <-- import de tqdm

# 1) Choix du device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2) Chargement du modèle
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
model.to(device)

# (Optionnel) geler le backbone visuel au début
for param in model.vision_model.parameters():
    param.requires_grad = False

# 3) Optimiseur + scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
total_steps = len(train_loader) * num_epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = int(0.1 * total_steps),
    num_training_steps = total_steps
)

# 4) Boucle d'entraînement avec tqdm
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # tqdm pour la phase train
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
    for batch in train_bar:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(
            pixel_values   = batch["pixel_values"],
            input_ids      = batch["input_ids"],
            attention_mask = batch["attention_mask"],
            labels         = batch["labels"]
        )
        loss = outputs.loss
        running_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # mise à jour du descriptif avec la loss courante
        train_bar.set_postfix(loss=running_loss / (train_bar.n + 1))

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} — Loss entraînement: {avg_train_loss:.4f}")

    # 5) Évaluation sur validation avec tqdm
    if 'val_loader' in globals():
        model.eval()
        correct, total = 0, 0
        val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
        with torch.no_grad():
            for batch_idx, batch in enumerate(val_bar):
                batch = {k: v.to(device) for k, v in batch.items()}

                # Generate predictions using the 'generate' method
                generated_ids = model.generate(
                    pixel_values=batch["pixel_values"],
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"]
                )

                # Decode the generated IDs to get the predicted answers
                preds = processor.batch_decode(generated_ids, skip_special_tokens=True)

                # Get the ground truth answers from the validation dataset
                labels = val_dataset.ds[batch_idx * val_loader.batch_size : (batch_idx + 1) * val_loader.batch_size]["answers"]
                labels = [ans[0] for ans in labels] # Get the first answer of each

                # Calculate accuracy based on predicted and ground truth answers
                correct += sum([pred == label for pred, label in zip(preds, labels)])
                total += len(preds)

                # mise à jour du descriptif avec l'accuracy partielle
                val_bar.set_postfix(acc=correct / total)

        acc = correct / total
        print(f"→ Accuracy validation: {acc:.4f}")

Epoch 1/3 [Train]:   0%|          | 0/1283 [00:00<?, ?it/s]

Epoch 1/3 — Loss entraînement: 1.5474


Epoch 1/3 [Val]:   0%|          | 0/270 [00:00<?, ?it/s]

→ Accuracy validation: 0.2371


Epoch 2/3 [Train]:   0%|          | 0/1283 [00:00<?, ?it/s]

Epoch 2/3 — Loss entraînement: 0.3490


Epoch 2/3 [Val]:   0%|          | 0/270 [00:00<?, ?it/s]

→ Accuracy validation: 0.1966


Epoch 3/3 [Train]:   0%|          | 0/1283 [00:00<?, ?it/s]

In [ ]:
from PIL import Image
import torch

def predict_vqa(image_path: str,
                question: str,
                model: BlipForQuestionAnswering,
                processor: BlipProcessor,
                device: torch.device) -> str:
    """
    Prédit la réponse à une question sur une image avec un modèle BLIP-VQA.

    Args:
        image_path (str): chemin vers le fichier image.
        question   (str): texte de la question.
        model      (BlipForQuestionAnswering): modèle finetuné.
        processor  (BlipProcessor): processor associé.
        device     (torch.device): 'cuda' ou 'cpu'.

    Returns:
        str: réponse générée par le modèle.
    """
    model.eval()
    # Chargement et prétraitement
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image,
                       question,
                       return_tensors="pt").to(device)

    # Génération
    with torch.no_grad():
        generated_ids = model.generate(**inputs,
                                       max_length=16,
                                       num_beams=5,
                                       early_stopping=True)
    # Décodage
    answer = processor.batch_decode(generated_ids,
                                    skip_special_tokens=True)[0]
    return answer


In [ ]:
# supposons que model, processor et device sont déjà définis
img_path = "/content/apple.png"
question = "What is the name of the object in the image?"
response = predict_vqa(img_path, question, model, processor, device)
print("Réponse :", response)


Réponse : apple


In [ ]:
# supposons que model, processor et device sont déjà définis
img_path = "/content/apple.png"
question = "What is the color ?"
response = predict_vqa(img_path, question, model, processor, device)
print("Réponse :", response)


Réponse : green


In [ ]:
# supposons que model, processor et device sont déjà définis
img_path = "/content/cat.jpg"
question ="is it a dog"
response = predict_vqa(img_path, question, model, processor, device)
print("Réponse :", response)


Réponse : no


In [ ]:
# supposons que model, processor et device sont déjà définis
img_path = "/content/cat.jpg"
question ="is it a cat"
response = predict_vqa(img_path, question, model, processor, device)
print("Réponse :", response)


Réponse : yes
